## 2/11/2017 :

I started by cloning/downloading the git repository of openCitations locally and start analyzing the general structure/organization of the project. My main attention today was on the ‘website/‘ directory which defines the web pages of OpenCitations. Starting from the server module ‘oc.py’, I studied and commented the code and its main parts. This analyzes was essential to better understand web.py since I have never used it before, and the code itself. 
<br /><br />
Once I have understood the basic/essential parts, I re-adapted the configuration file according to my local URLs. All the necessary modules to run the python server were successfully installed under the python2.7 version. The website runs correctly locally and I can use it to formulate some tests on it.

## 3/11/2017